# **Hugging Face Transformer Model For Machine Translation**
- Download **Helsinki-NLP/opus-mt-en-hi** **transformer** based **LLM** model from hugging face
- Here we convert from english to hindi
- Do the **Fine tuning** of that pretrained model using our dataset **cfilt/iitb-english-hindi**
- Do **Machine Translation**  Using finetuned model
- Use the Fine tuned moddel and do the inferencing on new data to identify **English to Hindi Trnslation** in that data


- **cfilt/iitb-english-hindi** - The IIT Bombay English-Hindi corpus contains parallel corpus for English-Hindi as well as monolingual Hindi corpus collected from a variety of existing sources and corpora developed


- Here we building our own translation system -  **language translation system**

- from the hugging face we are going to use open source translation model

- data set also i wll take from hugging face

- Here **Tensorflow** method used, it WONT take care of all **compilation optimize** etc,  need to define compilation adam optimizer. In case of **Pytourch /torch** , It will take care

##### 3 Ways we can use pretrained  Model

-  Transfer learning (Direct use without Fine tune or 1st we do transfer learning(use existing weight) of that model and then fine tune)

- FineTuning(for this translaiton system we gonnda perform finetune of pretrain model)

- RAG System(will implement it in my future class-> as of now dont focous here)

#### Different Machine/Language Trnslation models and API's

- Google- T5

- Facebook- M2M100

- Helinsinki-NLP

- Google Cloud Translaiton API

- Tune AI

- Amazone Comprehend

In [ ]:
!nvidia-smi

Fri Mar  1 06:00:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0              28W /  70W |  14065MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.27.2
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
be

## Downloading Libraries

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
#Here transformers means Hugging face model hub only

In [ ]:
!pip install --upgrade accelerate -q
!pip uninstall -y transformers accelerate -q
!pip install transformers accelerate -q

In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM  # Here Tensorflow(TF) BASED AutoModelForSeq2SeqLM Used
from transformers import DataCollatorForSeq2Seq
from transformers import AdamWeightDecay  #Used Adam optimizer

## Downloading Hugging face **Helsinki-NLP/opus-mt-en-hi** model

### 1. Use Pretrained Model - **Helsinki-NLP/opus-mt-en-hi**'s Tokenizer
### 2. Load/download Pretrained Model - **Helsinki-NLP/opus-mt-en-hi** itself

In [ ]:
model="Helsinki-NLP/opus-mt-en-hi"

# Load the tokens from this model, we will get all weights to make the tokens
tokenizer = AutoTokenizer.from_pretrained(model)

#Load pretrained model to Cuda device
model=TFAutoModelForSeq2SeqLM.from_pretrained(model)
#model=TFAutoModelForSeq2SeqLM.from_pretrained(model).to(device)#cuda
model

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


## Downloading Hugging face **cfilt/iitb-english-hindi** data for Fine tune above model

- **cfilt/iitb-english-hindi** is a corpus with english to Hindi trnslation

In [ ]:
dataset=load_dataset("cfilt/iitb-english-hindi")
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

> It has data with only 1 field/feature as translation. Which contains  data as dictionary (both english and hindi)

In [ ]:
dataset["train"][0]
#In translation field only both english and hindi available as dictionary

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [ ]:
dataset["train"][1]

{'translation': {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}}

In [ ]:
dataset["validation"]

Dataset({
    features: ['translation'],
    num_rows: 520
})

In [ ]:
dataset["validation"][0]

{'translation': {'en': 'Students of the Dattatreya city Municipal corporation secondary school demonstrated their imagination power by creating the fictitious fort "Duttgarh".',
  'hi': "महानगर पालिका अंतर्गत दत्तात्रय नगर माध्यमिक स्कूल के विद्यार्थियों ने काल्पनिक किला 'दत्तगढ़' बनाकर अपनी कल्पनाशक्ति का परिचय दिया।"}}

In [ ]:
dataset["test"]

Dataset({
    features: ['translation'],
    num_rows: 2507
})

In [ ]:
dataset["test"][5]

{'translation': {'en': 'The tea party is aghast.',
  'hi': 'चाय पार्टी भौचक्की है।'}}

In [ ]:
dataset["validation"]["translation"][:5]

[{'en': 'Students of the Dattatreya city Municipal corporation secondary school demonstrated their imagination power by creating the fictitious fort "Duttgarh".',
  'hi': "महानगर पालिका अंतर्गत दत्तात्रय नगर माध्यमिक स्कूल के विद्यार्थियों ने काल्पनिक किला 'दत्तगढ़' बनाकर अपनी कल्पनाशक्ति का परिचय दिया।"},
 {'en': 'With encouragement from Principal Sandhya Medpallivaar the teachers and students built the fort out of clay.',
  'hi': 'प्रधानाध्यापक संध्या मेडपल्लीवार के प्रोत्साहित करने पर शिक्षकों व विद्यार्थियों ने मिट्टïी से किले का निर्माण किया।'},
 {'en': 'Rajesh Gavre, the President of the MNPA teachers association, honoured the school by presenting the award.',
  'hi': 'मनपा शिक्षक संघ के अध्यक्ष राजेश गवरे ने स्कूल को भेंट देकर सराहना की।'},
 {'en': 'Ramesh Saatpute examined the fort.',
  'hi': 'किले का परीक्षण रमेश सातपुते ने किया।'},
 {'en': 'Students like Nikhil Kavle, Darshan Gedekar, Sahil Meshram participated in building the fort.',
  'hi': 'किला निर्माण में निखिल कावले, दर

In [ ]:
for ex in  dataset["validation"]["translation"][:5]:
  print(ex["en"])

Students of the Dattatreya city Municipal corporation secondary school demonstrated their imagination power by creating the fictitious fort "Duttgarh".
With encouragement from Principal Sandhya Medpallivaar the teachers and students built the fort out of clay.
Rajesh Gavre, the President of the MNPA teachers association, honoured the school by presenting the award.
Ramesh Saatpute examined the fort.
Students like Nikhil Kavle, Darshan Gedekar, Sahil Meshram participated in building the fort.


In [ ]:
for ex in  dataset["validation"]["translation"][:5]:
  print(ex["hi"])

महानगर पालिका अंतर्गत दत्तात्रय नगर माध्यमिक स्कूल के विद्यार्थियों ने काल्पनिक किला 'दत्तगढ़' बनाकर अपनी कल्पनाशक्ति का परिचय दिया।
प्रधानाध्यापक संध्या मेडपल्लीवार के प्रोत्साहित करने पर शिक्षकों व विद्यार्थियों ने मिट्टïी से किले का निर्माण किया।
मनपा शिक्षक संघ के अध्यक्ष राजेश गवरे ने स्कूल को भेंट देकर सराहना की।
किले का परीक्षण रमेश सातपुते ने किया।
किला निर्माण में निखिल कावले, दर्शन गेड़ेकर, साहिल मेश्राम इन विद्यार्थियों ने सहभाग लिया।


## Apply this pretrained model tokenizer on  dataset to convert text to vector
- It converts Data/text to integer/number, based on Helsinki Tokenizer dictonary/pretrained embedded model
- Once we apply tokenizer, it creats dictionary with **'input_ids' and 'attention_mask'** for that sentence

- To we have different tokenizer set for **Input/english and Target/Hindi**
- So we need to use **tokenizer** for **input** sentences and **tokenizer.as_target_tokenizer()** for output sentences it holds taget specific tokens

In [ ]:
tokenizer("there is sunny who is teachine data science")

{'input_ids': [188, 23, 41670, 66, 23, 2144, 6131, 1195, 7373, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(['चाय पार्टी भौचक्की है।'])

{'input_ids': [[44, 4228, 260, 1321, 44, 2703, 260, 428, 1185, 2136, 130, 44, 1, 1185, 716, 130, 44, 1, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
  print(tokenizer(['चाय पार्टी भौचक्की है।']))

{'input_ids': [[15889, 9384, 61313, 130, 5, 40, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
max_input_length = 128
max_target_length = 128

source_lang="en"
target_lang="hi"

def process_function(sentence):

  #Convert all english sentences inside 1 list
  inputs=[ex[source_lang] for ex in sentence["translation"]]

  #Convert all Hindi sentences inside 1 list
  targets=[ex[target_lang] for ex in sentence["translation"]]

  #Apply Tokenizer to convert english/Input to vector
  model_inputs=tokenizer(inputs,max_length=max_input_length,truncation=True)

  #Apply Tokenizer to convert hindi/Target to vector
  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets,max_length=max_target_length,truncation=True)


  model_inputs["labels"]=labels["input_ids"]
  return model_inputs


In [ ]:
# Used process_function to convert word to embedding for all 3 data(Train/dtest/validation)
#Here we pass data in batch, which part of pytorch. It appends new features to existing data
#Here map passes train, validation,test data as separate batch  and applies process_function on this
tokenized_data=dataset.map(process_function, batched=True)

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2507
    })
})

> After this tokenizer, it adds these 3 features to data - **'input_ids', 'attention_mask', 'labels'**

In [ ]:
tokenized_data["train"]

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1659083
})

In [ ]:
tokenized_data["train"]["labels"][0]

[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0]

## Fine tune the model

In [ ]:
model

#### Define parameter and datacolletor

In [ ]:
#Define some parameter
batch_size=32
learning_rate=0.001  ## Required for adam optimizer
weight_decay=0.01  ## Required for adam optimizer
num_train_epochs=1

In [ ]:
#Define data Collatoe
data_collator=DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf") #We are using tensor flow framework

#### Prepare train and Validation Tensorflow dataset

In [ ]:
train_dataset=model.prepare_tf_dataset(
    tokenized_data["test"],  # Here train data is huge, so using test data as training set
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator
)

In [ ]:
validation_dataset=model.prepare_tf_dataset(
    tokenized_data["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

#### Define adam optimizer

In [ ]:
optimizer=AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)

#### Compile and Fit(This part is fine tuning) the model

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
#this is my finetuing
model.fit(train_dataset,validation_data=validation_dataset,epochs=2)

Epoch 1/2
78/78 [==============================] - 118s 765ms/step - loss: 4.6010 - val_loss: 4.7798
Epoch 2/2
78/78 [==============================] - 46s 592ms/step - loss: 3.3207 - val_loss: 4.9300


## Save the Fine tuned Model in tf_model folder in local

In [ ]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


## Invoke/Call fine tuned new model

#### Model input Tokenizer

In [ ]:
input_text="Thousands of motorists have already taken the black boxes, some of which have GPS monitoring, for a test drive."

In [ ]:
tokenized=tokenizer([input_text],return_tensors='np') #here np means numpy
tokenized

{'input_ids': array([[15385,     8, 19396,  9810,    55,   927,  1233,     4,  3078,
        18743,     2,   241,     8,   157,    55, 22238, 38940,     2,
           34,    19,  2564,  3010,     3,     0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]])}

#### Model inference

In [ ]:
out=model.generate(**tokenized, max_length=128) #If we want to pass dictionary argument we use **
out[0]

<tf.Tensor: shape=(41,), dtype=int32, numpy=
array([61949, 21457, 21682,    69, 26901,   716,     6,   173,   964,
         169,    58, 20501,  8323,     5,     2,   994,   602,     3,
        6157,     3,  6157,     3,  5674,     3,     6,   173,  7268,
           6,    39,   176,    12,    99,    18,   204,    58,   237,
         324,     5,    40,     0, 61949], dtype=int32)>

#### Model inferenced vector Decode

In [ ]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0],skip_special_tokens=True)) #This skip_special_tokens will skips pad/white space at the end


हजारों मोटर-चालक के पास अभी तक ही ब्लैक बॉक्स है, जिसमें जी.पी.पी.एस. के पास परीक्षण के लिए बहुत से लोगों को पहले ही ले लिया है।


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## Evaluate the Model

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Refer this link for better coding and  detail on trnslation
- https://huggingface.co/docs/transformers/tasks/translation

# END